### **Data Reading**

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

In [0]:
df = spark.read.format("parquet") \
          .load("abfss://bronze@databricksend2end.dfs.core.windows.net/orders")
display(df)          

In [0]:
df.printSchema()

In [0]:
df = df.drop("_rescued_data")
display(df)

In [0]:
df = df.withColumn("order_date", to_timestamp(col("order_date")))
display(df)

In [0]:
df = df.withColumn("year", year(col("order_date")))
display(df)

In [0]:
df1 = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

In [0]:
df1 = df1.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

In [0]:
df1 = df1.withColumn("row_flag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
display(df1)

### **Classes- OOPs Concept**

In [0]:
class windows:

  def dense_rank(self, df):
    df_dense_rank = df.withColumn("flag",dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    return df_dense_rank
  
  def rank(self, df):
    df_rank = df.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    return df_rank
  
  def row_number(self,df):
    df_row_number = df.withColumn("row_flag",row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
    return df_row_number

In [0]:
df_new = df

In [0]:
display(df_new)

In [0]:
obj = windows()

In [0]:
df_results = obj.dense_rank(df)
display(df_results)

### **Data Writing**

In [0]:
df.write.format("delta").mode("overewrite").save("abfss://silver@databricksend2end.dfs.core.windows.net/orders")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS databricks_cata.silver.orders
USING DELTA
LOCATION 'abfss://silver@databricksend2end.dfs.core.windows.net/orders'

In [0]:
%sql
SELECT * FROM databricks_cata.silver.orders